In [122]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [102]:
suicide2005=pd.read_csv("2005.csv")
suicide2006=pd.read_csv("2006.csv")
suicide2007=pd.read_csv("2007.csv")
suicide2008=pd.read_csv("2008.csv")
suicide2009=pd.read_csv("2009.csv")
suicide2010=pd.read_csv("2010.csv")
suicide2011=pd.read_csv("2011.csv")

In [103]:
suicide2007.columns

Index(['Unnamed: 0', 'state', 'vet_pop', 'overall_pop_18', 'vet_pop_p',
       'vet_suicides', 'all_suicides', 'vet_suicides_p', 'vet_males',
       'vet_males_p', 'vet_females', 'vet_females_p', 'vet_15_24', 'vet_18_29',
       'vet_17_34', 'vet_25_34', 'vet_30_39', 'vet_25_44', 'vet_35_44',
       'vet_40_49', 'vet_35-54', 'vet_45_54', 'vet_50_59', 'vet_45_64',
       'vet_55_64', 'vet_60', 'vet_65', 'vet_rate', 'civ_rate'],
      dtype='object')

In [104]:
frames = [suicide2005, suicide2006, suicide2007, suicide2008, suicide2009, suicide2010, suicide2011]
bigdata = pd.concat(frames)
bigdata.head()
bigdata.to_csv("Cumulated_Data.csv", index=False)


In [112]:
Y = bigdata.vet_suicides
feature_names = ["state", "vet_pop", 
                 "vet_males", "vet_females"]

X = bigdata[feature_names]
print(X)

             state  vet_pop  vet_males  vet_females
0          Alabama   403950        NaN          NaN
1           Alaska    74482       22.0          2.0
2          Arizona   538880        NaN          NaN
3         Arkansas   259304       79.0          2.0
4       California  2193336      613.0         20.0
5         Colorado   402091      183.0          8.0
6      Connecticut   261294        NaN          NaN
7         Delaware    79151       17.0          0.0
8         Florida*  1717801        NaN          NaN
9          Georgia   731466        NaN          NaN
10          Hawaii   116793        NaN          NaN
11           Idaho   132844       63.0          2.0
12        Illinois   853338        NaN          NaN
13         Indiana   505259      111.0          1.0
14            Iowa   249911        NaN          NaN
15          Kansas   238506       82.0          4.0
16        Kentucky   341752        NaN          NaN
17       Louisiana   337822        NaN          NaN
18          

In [106]:
bigdata.head()

,Unnamed: 0,state,vet_pop,overall_pop_18,vet_pop_p,vet_suicides,all_suicides,vet_suicides_p,vet_males,vet_males_p,...,vet_40_49,vet_35-54,vet_45_54,vet_50_59,vet_45_64,vet_55_64,vet_60,vet_65,vet_rate,civ_rate
0,0,Alabama,403950,3344721,0.120772,135,535,0.252336,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.419978,13.601875
1,1,Alaska,74482,446969,0.166638,24,131,0.183206,22.0,0.916667,...,NaN,NaN,9.0,NaN,NaN,5.0,NaN,4.0,32.222550,28.725835
2,2,Arizona,538880,4238996,0.127124,225,945,0.238095,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.753266,19.458849
3,3,Arkansas,259304,2023819,0.128126,81,400,0.202500,79.0,0.975309,...,NaN,NaN,NaN,NaN,NaN,16.0,NaN,25.0,31.237466,18.078622
4,4,California,2193336,25543447,0.085867,633,3206,0.197442,613.0,0.968404,...,NaN,NaN,NaN,NaN,220.0,NaN,NaN,317.0,28.860147,11.019220


In [114]:
label_encoder = preprocessing.LabelEncoder()
X = X.apply(label_encoder.fit_transform)
X.head()

,state,vet_pop,vet_males,vet_females
0,0,201,182,149
1,1,35,9,2
2,2,261,208,19
3,3,151,48,2
4,4,349,140,15


In [115]:
X = X.fillna(0)
one_hot_encoded_training_parameters = pd.get_dummies(X)

In [116]:
print(X)

    state  vet_pop  vet_males  vet_females
0       0      201        182          149
1       1       35          9            2
2       2      261        208           19
3       3      151         48            2
4       4      349        140           15
5       5      200         91            8
6       6      153        209           29
7       7       43          7            0
8       8      341        210           22
9       9      288        211           21
10     10       62        212           61
11     11       77         37            2
12     12      307        213           35
13     13      249         64            1
14     14      146        214           57
15     15      139         51            4
16     16      176        215           56
17     17      173        216           55
18     18       85        217           53
19     19      236        218           52
20     20      227         28            2
21     21      304        114            7
22     22  

In [117]:
X.isnull().any()

state          False
vet_pop        False
vet_males      False
vet_females    False
dtype: bool

In [118]:
suicide_model = DecisionTreeRegressor(random_state=1)
suicide_model.fit(X,Y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [119]:
predictions = suicide_model.predict(X)
print(predictions)

[135.  24. 225.  81. 633. 191.  52.  17. 540. 182.  15.  65. 219. 112.
  70.  86.  44. 101.  49. 105.  51. 232. 103.  68. 180.  59.  18. 106.
  37.  80.  80. 208. 225.  24. 282. 109. 154. 336.  15.  96.  29. 183.
 509.  49.  10. 206. 225.  73. 136.  17. 145.  29. 225.  90. 665. 148.
  44.  16. 562.  97.  21.  53. 186. 108.  63.  94.  34.  81.  43.  91.
  30. 258. 113.  65. 180.  54.   5. 107.  33.  93.  76. 216. 200.  18.
 251. 113. 143. 309.  21. 118.  26. 150. 454.  63.  16. 178. 200.  56.
 138.  22. 130.  30. 224.  84. 655. 196.  39.  15. 595. 211.  21.  48.
 183. 101.  63.  88.  44.  91.  39.  76.  50. 230. 125.  83. 180.  53.
  15. 104.  42.  80.  96. 205. 229.  27. 275. 112. 157. 324.  16. 123.
  21. 182. 434.  68.  19. 178. 229.  67. 128.  27. 138.  27. 233.  96.
 727. 188.  46.  23. 631. 192.  20.  75. 229. 184.  74.  74.  51.  97.
  44.  92.  48. 224. 110.  81. 180.  50.  21. 131.  36.  77.  80. 224.
 238.  14. 264. 145. 140. 333.  16. 137.  20. 176. 454.  55.  15. 203.
 238. 

In [121]:
predicted_suicide_rates = suicide_model.predict(X)
mean_absolute_error(Y, predicted_suicide_rates)

0.0

In [124]:
train_X, val_X, train_Y, val_Y = train_test_split(X,Y,random_state=0)
suicide_model.fit(train_X, train_Y)
value_of_predictions = suicide_model.predict(val_X)
print(mean_absolute_error(val_Y, value_of_predictions))

11.704545454545455
